In [1]:

import os
import json
import glob
import copy
import numpy as np
import pylab as pl
import seaborn as sns
import pandas as pd


from scipy import stats as spstats

from pipeline.python.classifications import experiment_classes as util
from pipeline.python.utils import natural_keys, label_figure

/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/2p-pipeline/pipeline/python/classifications/experiment_classes.py:18: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/traitlets/config/applica

In [2]:
%matplotlib notebook

In [3]:
def reformat_morph_values(sdf):
    control_ixs = sdf[sdf['morphlevel']==-1].index.tolist()
    sizevals = np.array([round(s, 1) for s in sdf['size'].unique() if s not in ['None', None] and not np.isnan(s)] )
    sdf.loc[sdf.morphlevel==-1, 'size'] = pd.Series(sizevals, index=control_ixs)
    sdf['size'] = [round(s, 1) for s in sdf['size'].values]

    return sdf


In [4]:
def test_blob_conditions(sdf, n_expected_size=5, n_expected_morph=10, param_kws=None):

    tested_sizes = np.array(sorted(sdf['size'].unique()))
    tested_morphs = np.array(sorted(sdf['morphlevel'].unique()))

    n_sizes = len(tested_sizes)
    n_morphs = len(tested_morphs)

    all_correct = (n_sizes == n_expected_size and n_morphs == n_expected_morph)
    return all_correct

def test_gratings_conditions(sdf, params={'ori': tuple(np.arange(0, 360, 45)),
                                          'speed': (10, 20),
                                          'sf': (0.1, 0.5),
                                          'size_full': 200,
                                          'size_aperture': 30}, param_kws=None):
    '''
    Use kwargs={stimparam: stimvalue} to update default values.
    Include kwargs{'ignore': stimparam} to ignore params.
    '''
    check = {}
    if param_kws is not None:
        for k, v in param_kws.iteritems():
            if k=='ignore' and v in params.keys():
                print("ignoring param: %s" % v)
                params.pop(v)
            else:
                params.update({k: v})
    
    for param, paramvalue in params.items():
        if param == 'size_full' or 'size_aperture':
            tested_values = tuple(sorted(sdf['size'].unique()))
            if param == 'size_full':
                check[param] = (paramvalue, max(tested_values) == paramvalue)
            elif param == 'size_aperture':
                check[param] = (paramvalue, min(tested_values) == paramvalue)
        else:
            check[param] = (paramvalue, tested_values == sorted(paramvalue))
           
    if all([c[1] for k, c in check.items()]):
        return True
    else:
        print check
        return False
    
    
def check_stimulus_conditions(sdf, experiment_type='blobs', param_kws=None):
    if experiment_type=='blobs':
        all_correct = test_blob_conditions(sdf, param_kws=param_kws)
    elif experiment_type=='gratings':
        all_correct = test_gratings_conditions(sdf, param_kws=param_kws)
    else:
        print("Not implemented! %s (including all conds)" % experiment_type)
        return True
    
    return all_correct


In [5]:
# rootdir = '/n/coxfs01/2p-data'
# animalid = 'JC084' #'JC059'
# session = '20190522' #'20190227'
# fov = 'FOV1_zoom2p0x' #'FOV4_zoom4p0x'
# traceid = 'traces001' #'traces001'
# response_type = 'dff'


# Get aggregated data

In [6]:
from pipeline.python.classifications import get_dataset_stats as gd
import cPickle as pkl

options = ['-t', 'traces001']
optsE = gd.extract_options(options)

rootdir = optsE.rootdir
aggregate_dir = optsE.aggregate_dir
fov_type = optsE.fov_type
traceid = optsE.traceid
trace_type = optsE.trace_type
response_type = 'dff'
print aggregate_dir

/n/coxfs01/julianarhee/aggregate-visual-areas


/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/2p-pipeline/pipeline/python/classifications/get_dataset_stats.py:17: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to u'nbAgg' by the following code:
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_inst

In [7]:
sdata_fpath = os.path.join(aggregate_dir, 'dataset_info.pkl')
if os.path.exists(sdata_fpath):
    with open(sdata_fpath, 'rb') as f:
        sdata = pkl.load(f)
else:
    sdata = gd.aggregate_session_info(traceid=optsE.traceid, trace_type=optsE.trace_type, 
                                       state=optsE.state, fov_type=optsE.fov_type, 
                                       visual_areas=optsE.visual_areas,
                                       blacklist=optsE.blacklist, 
                                       rootdir=optsE.rootdir)
    with open(sdata_fpath, 'wb') as f:
        pkl.dump(sdata, f, protocol=pkl.HIGHEST_PROTOCOL)


# Set responsivity params

In [8]:
responsive_test = 'nstds'
responsive_thr = 10.
n_stds = 2.5


# Get aggregate stats

In [9]:
# Set plotting params
threshold_dff = True
min_dff = 0.03
thresh_str = 'thr-%.2f' % min_dff if threshold_dff else 'no-thr'


In [10]:
stats_desc = util.get_stats_desc(traceid=optsE.traceid,
                                      trace_type= optsE.trace_type,
                                      response_type = response_type,
                                      responsive_test = responsive_test,
                                      responsive_thr = responsive_thr,
                                      n_stds = n_stds)
print(stats_desc)

stats-traces001-corrected-dff-nstds-thr-10.00


In [14]:
create_new = False

# Create aggregate stat files

In [15]:
for experiment_type in ['blobs', 'gratings', 'rfs']:
    
    dsets = sdata[sdata['experiment']==experiment_type]


    if experiment_type == 'gratings':
        param_kws = {'ignore': 'size_aperture'}
    else:
        param_kws = None

    # Set output dirs
    aggr_tcourse_dir = os.path.join(aggregate_dir, 'data-stats', 'experiment-objects', experiment_type)
    aggr_stats_dir = os.path.join(aggregate_dir, 'data-stats', stats_desc)

    if not os.path.exists(aggr_tcourse_dir):
        os.makedirs(aggr_tcourse_dir)
    if not os.path.exists(aggr_stats_dir):
        os.makedirs(aggr_stats_dir)
    print("Experiment objects:\n--->%s" % aggr_tcourse_dir)
    print("Experiment stats:\n--->%s" % aggr_stats_dir)


    for visual_area in ['V1', 'Lm', 'Li']:

        sessions = dsets[ dsets['visual_area'] == visual_area ]
        n_datasets = sessions.shape[0]

        sfilename = os.path.join(aggr_stats_dir, '%s_%s_%s-%s.pkl' % (visual_area, experiment_type, traceid, response_type))
        print("Stats: %s" % sfilename)
        allstats = {}

        incorrect_params = []
        for (animalid, session, fov), g in sessions.groupby(['animalid', 'session', 'fov']):

            skey = '-'.join([animalid, session, fov])

            if experiment_type == 'gratings':
                exp = util.Gratings(animalid, session, fov, traceid=traceid, rootdir=rootdir)    
            elif experiment_type == 'blobs':
                exp = util.Objects(animalid, session, fov, traceid=traceid, rootdir=rootdir)    
            elif experiment_type == 'rfs':
                exp = util.ReceptiveFields(experiment_type, animalid, session, fov, traceid=traceid, rootdir=rootdir)

            # Get stimulus conditions
            sdf = exp.get_stimuli()
            if experiment_type == 'blobs':
                sdf = reformat_morph_values(sdf)

            # Test stimulus conditions are correct
            all_correct = check_stimulus_conditions(sdf, experiment_type=experiment_type, param_kws=param_kws)            
            if not all_correct:
                incorrect_params.append(skey)
                continue

            trace_desc = '%s_%s-%s' % (skey, traceid, trace_type)
            tfilename = os.path.join(aggr_tcourse_dir, '%s_%s.pkl' % (visual_area, trace_desc))
            print("Timecourse: %s" % os.path.split(tfilename)[-1])
            load_data=True
#             if not create_new:
#                 try:
#                     with open(tfilename, 'rb') as f:
#                         exp = pkl.load(f)
#                         assert exp.data is not None and 'traces' in dir(exp.data)
#                         sdf = exp.data.sdf.copy()
#                 except Exception as e:
#                     load_data = True
#             else:
#                 load_data = True

            if load_data:
                # Get metrics for each condition
                estats = exp.get_stats(responsive_test=responsive_test, \
                                       responsive_thr=responsive_thr, n_stds=n_stds)
                if exp.data is None:
                    exp.load(trace_type=trace_type, make_equal=True)
                print(exp.data.traces.shape)
                with open(tfilename, 'wb') as f:
                    pkl.dump(exp, f, protocol=pkl.HIGHEST_PROTOCOL)
                allstats[skey] = estats

        with open(sfilename, 'wb') as f:
            pkl.dump(allstats, f, protocol=pkl.HIGHEST_PROTOCOL)

        if len(incorrect_params) > 0:
            with open(os.path.join(os.path.split(aggr_stats_dir)[0], '%s_%s_incorrect-params.json' % (visual_area, experiment_type)), 'w') as f:
                json.dump(incorrect_params, f)


Experiment objects:
--->/n/coxfs01/julianarhee/aggregate-visual-areas/data-stats/experiment-objects/blobs
Experiment stats:
--->/n/coxfs01/julianarhee/aggregate-visual-areas/data-stats/stats-traces001-corrected-dff-nstds-thr-10.00
Stats: /n/coxfs01/julianarhee/aggregate-visual-areas/data-stats/stats-traces001-corrected-dff-nstds-thr-10.00/V1_blobs_traces001-dff.pkl
... [JC076|20190420|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
... renaming experiment to run name: combined_blobs_static
Getting stimulus info for: combined_blobs_static
Timecourse: V1_JC076-20190420-FOV1_zoom2p0x_traces001-corrected.pkl
... [combined_blobs_static] Loading roi stats and cell list...
... loading data array
/n/coxfs01/2p-data/JC076/20190420/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_5ffca7_traces001_09c921_traces001_512e96_traces001_394d78_traces001_64e684_traces001_6bacb1/data_arrays/np_subtracted.npz
adding NP offset...
... updating self
filtering responsive cells: n

adding NP offset...
... adjusting for equal reps / condn...
... updating self
filtering responsive cells: nstds
... getting responsive cells (test: nstds, thr: 10.00')
(259550, 293)
... [JC078|20190504|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
... renaming experiment to run name: combined_blobs_static
Getting stimulus info for: combined_blobs_static
Timecourse: Lm_JC078-20190504-FOV1_zoom2p0x_traces001-corrected.pkl
... [combined_blobs_static] Loading roi stats and cell list...
... loading data array
/n/coxfs01/2p-data/JC078/20190504/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_830af1_traces001_241385_traces001_4f577b_traces001_8a5bd2_traces001_af220b_traces001_f47906/data_arrays/np_subtracted.npz
adding NP offset...
... updating self
filtering responsive cells: nstds
... getting responsive cells (test: nstds, thr: 10.00')
(268500, 267)
... [JC078|20190509|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
... renaming expe

Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 7 (has 3438 frames)
... asking for 35 extra frames...
... Last frame to align: 3473 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 3350)
N frames per trial: 134
N tstamps: 3350
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -93.70632737276492
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -389.1468475545124
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -303.85004882812495
STIM TYPE: image
**** File 9 of 59 *****
*** Excluding: []
/n/coxfs01/2p-data/JC078/20190513/FOV1_zoom2p0x/blobs_run1
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 8 (has 3438 frames)
... asking for 36 extra frames...


Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 7 (has 3438 frames)
... asking for 35 extra frames...
... Last frame to align: 3473 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 3350)
N frames per trial: 134
N tstamps: 3350
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -77.56430594900871
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -319.26798002073497
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -313.0815429687498
STIM TYPE: image
**** File 19 of 59 *****
*** Excluding: []
/n/coxfs01/2p-data/JC078/20190513/FOV1_zoom2p0x/blobs_run2
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 8 (has 3438 frames)
... asking for 35 extra frames...

Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 7 (has 3438 frames)
... asking for 35 extra frames...
... Last frame to align: 3473 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 3350)
N frames per trial: 134
N tstamps: 3350
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -90.51704545454584
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -294.2910157575054
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -331.355261600379
STIM TYPE: image
**** File 29 of 59 *****
*** Excluding: []
/n/coxfs01/2p-data/JC078/20190513/FOV1_zoom2p0x/blobs_run3
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 3350
... N unique frames: 3350
... File: 8 (has 3438 frames)
... asking for 35 extra frames...
.

...

Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -150.94988986784142
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -294.47255995214493
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -324.51740486391134
STIM TYPE: image
**** File 34 of 60 *****
*** Excluding: []
/n/coxfs01/2p-data/JC083/20190517/FOV1_zoom2p0x/blobs_run4
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 4475
... N unique frames: 3447
... File: 3 (has 3438 frames)
... asking for 57 extra frames...
... Last frame to align: 3495 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 4475)
N frames per trial: 179
N tstamps: 4475
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -76.33908045977012
Showing initial drift correction (quantile: 0.10)
Min valu

Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -181.80705705705714
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -520.9589687740911
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -483.489919354839
STIM TYPE: image
**** File 44 of 60 *****
*** Excluding: []
/n/coxfs01/2p-data/JC083/20190517/FOV1_zoom2p0x/blobs_run5
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 4475
... N unique frames: 3447
... File: 3 (has 3438 frames)
... asking for 57 extra frames...
... Last frame to align: 3495 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 4475)
N frames per trial: 179
N tstamps: 4475
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -159.97366688610919
Showing initial drift correction (quantile: 0.10)
Min value 

Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -188.42136752136764
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -708.1319912073537
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -667.5390467489922
STIM TYPE: image
**** File 54 of 60 *****
*** Excluding: []
/n/coxfs01/2p-data/JC083/20190517/FOV1_zoom2p0x/blobs_run6
N tsecs: 3438
There are 250 total trials across all .tif files.
Frame indices are NOT block indexed
... N frames to align: 4475
... N unique frames: 3448
... File: 3 (has 3438 frames)
... asking for 57 extra frames...
... Last frame to align: 3495 (N frames total, 3438)
... N tstamps: 3438
... padding trial tstamps array... (should be 4475)
N frames per trial: 179
N tstamps: 4475
N trials in block: 25
('Excluding:', [u'filehash', u'filepath'])
Showing initial drift correction (quantile: 0.10)
Min value for all ROIs: -107.15121668597976
Showing initial drift correction (quantile: 0.10)
Min value

Traceback (most recent call last):
  File "/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/2p-pipeline/pipeline/python/classifications/experiment_classes.py", line 1001, in load
    dset = np.load(soma_fpath)
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/numpy/lib/npyio.py", line 384, in load
    fid = open(file, "rb")
IOError: [Errno 2] No such file or directory: '/n/coxfs01/2p-data/JC083/20190517/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_0c1b62_traces001_654830_traces001_c16392_traces001_266bc9_traces001_0b327f/data_arrays/np_subtracted.npz'


AttributeError: 'NoneType' object has no attribute 'traces'

In [55]:
exp.data

In [89]:
os.listdir(aggr_stats_dir)

['Li_blobs_dff-thr-0.03.pkl',
 'V1_blobs_dff-thr-0.03.pkl',
 'Lm_blobs_dff-thr-0.03.pkl',
 'V1_gratings_dff-thr-0.03.pkl']